# Quantization
This notebook acts as an example of how to use the quantization techniques.

## Setup
* Import the necessary packages.
* Load a model.
* Load a dataset.
* Analyze performance of the model prior to quantization.

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import torch
import importlib
import inspect
import sys
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

# Add thesis package to path
sys.path.append("../")
sys.path.append("../src")

import src.general as general
import src.metrics as metrics
import src.evaluation as eval
import src.compression.quantization as quant
import src.dataset_models as data
import src.plot as plot


In [35]:
# Get device
device = general.get_device()

# Load the dataset
dataset = data.supported_datasets["MNIST"]

In [36]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"

# Load the model
model = torch.load(model_state, map_location=torch.device(device))

### Pre-Quantization Evaluation

In [37]:
# Evaluate model performance before quantization
original_results = eval.get_results(model, dataset)
plot.print_results(**original_results)

Test: 100%|██████████| 157/157 [00:01<00:00, 126.83it/s]

Test loss: 0.0770
Test score: 97.6712
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.077024
Score: 97.671178
Time per data point: 0.4932 ms
Model Size: 1.65 MB
Number of parameters: 431080
Number of FLOPs: -1
Number of MACs: 2307728


## Static Quantization
Post Training Static Quantization (PTQ) also pre-quantizes model weights but instead of calibrating activations on-the-fly, the clipping range is pre-calibrated and fixed (“static”) using validation data.

In [38]:
static_quantized_model = quant.static_quantization(model, dataset)

Calibrating...
Calibration complete.


In [46]:
general.test(static_quantized_model, dataset)

Test:   0%|          | 0/157 [00:00<?, ?it/s]


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Metal, Negative, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID].

MPS: registered at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484780698/work

In [45]:
# Evaluate model performance after static quantization
static_quantized_results = eval.get_results(static_quantized_model, dataset)
plot.print_before_after_results(original_results, static_quantized_results)

Test:   0%|          | 0/157 [00:00<?, ?it/s]


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Metal, Negative, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID].

MPS: registered at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484780698/work

## Dynamic Quantization
Here the model’s weights are pre-quantized; the activations are quantized on-the-fly (“dynamic”) during inference. 

Currently only Linear and Recurrent (LSTM, GRU, RNN) layers are supported for dynamic quantization.

In [40]:
dynamic_quantized_model = quant.dynamic_quantization(model)

In [41]:
dynamic_quantized_results = eval.get_results(dynamic_quantized_model, test_loader, criterion, metric)
plot.print_before_after_results(original_results, dynamic_quantized_results)

NameError: name 'test_loader' is not defined